In [1]:
import re
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def parse_log_file(file_path):
    with open(file_path, "r") as f:
        content = f.read()

    # Split entries by delimiter
    entries = content.strip().split("-------###-------")
    
    data = []
    
    for entry in entries:
        entry = entry.strip()
        if not entry:
            continue
        
        # Extract IMG
        img_match = re.search(r"IMG:\s*(.*)", entry)
        img = img_match.group(1) if img_match else None
        
        # Extract Predicted Class and Confidence
        pred_match = re.search(r"Predicted Class:\s*(.*?)\s*\(([\d.]+)\s*confidence\)", entry)
        pred_class = pred_match.group(1) if pred_match else None
        confidence = float(pred_match.group(2)) if pred_match else None
        
        # Extract probabilities
        probs = {}
        prob_matches = re.findall(r"Tomato___\w+: [\d.]+", entry)
        for pm in prob_matches:
            cls, val = pm.split(":")
            probs[cls.strip()] = float(val.strip())
        
        row = {
            "Image": img,
            "Predicted_Class": pred_class,
            "Confidence": confidence,
            **probs
        }
        data.append(row)
    
    df = pd.DataFrame(data)
    return df


In [7]:
if __name__ == "__main__":
    #file_path = "/home/ec2-user/SageMaker/tmp/tomato_predictions.txt" 
    file_path = "/home/ec2-user/SageMaker/tmp/pred_sep.txt" 
    df = parse_log_file(file_path)
    #print(df.to_string(index=False))
    
    # Optionally save as CSV
    #df.to_csv("/home/ec2-user/SageMaker/tmp/parsed_predictions2.csv", index=False)
    df.to_csv("/home/ec2-user/SageMaker/tmp/parsed_sep.csv", index=False)
    print('done')

done
